# **Question 1**
RA and Dec that is published is usually for the epoch of 2000, called J2000. Do you think the RA and Dec would change today? Use astropy to find the RA and Dec of a radio source – Cygnus - A (RA 19h 59m 28.3566s  , Dec +40° 44′ 02.096″)

In [1]:
!pip install astropy -q

In [2]:
from astropy.coordinates import SkyCoord, FK5
from astropy.time import Time
import astropy.units as u

## Putting coordinates of cygnus A on epoch J200

In [3]:
coord_j2000 = SkyCoord(
    ra="19h59m28.3566s",
    dec="+40d44m02.096s",
    frame="icrs"
)

## Inputting date and time right now (after converting to UTC)
(timing middle of class)

In [4]:
t_today = Time("2025-01-09 05:00:00", scale="utc")
print(t_today)

2025-01-09 05:00:00.000


## Converting the coordinates from J2000 to today

In [5]:
coord_today = coord_j2000.transform_to(FK5(equinox=t_today))

In [6]:
print(f"J2000:\n"
    f"  RA  = {coord_j2000.ra.to_string(unit=u.hour, sep=':')} (h m s)\n"
    f"  Dec = {coord_j2000.dec.to_string(unit=u.deg,  sep=':')} (deg ' \")\n"
    f"Today:\n"
    f"  RA  = {coord_today.ra.to_string(unit=u.hour, sep=':')} (h m s)\n"
    f"  Dec = {coord_today.dec.to_string(unit=u.deg,  sep=':')} (deg ' \")"
)

J2000:
  RA  = 19:59:28.3566 (h m s)
  Dec = 40:44:02.096 (deg ' ")
Today:
  RA  = 20:00:20.33469845 (h m s)
  Dec = 40:48:12.6872363 (deg ' ")


Purely due to rotation and revolution of Earth there shouldn't be any change, but due to **'precision of Earth'** (wobbling of eath's rotation axis), the coordinates change slightly.  
Precession rate ≈ 50 arcsec/year  
25 years from 2000 to 2025 => ≈ 0.35° approx

# Question 2
What will be this source's azimuth and elevation when observing from IIT Kanpur?

In [7]:
from astropy.coordinates import AltAz, EarthLocation

## Entering location of IITK and height (sea level)

In [8]:
IITK = EarthLocation(lat=26.5*u.deg, lon=80.2*u.deg, height=128*u.m)

## Transofrming Equitorial coordinates to Alt Az for IITK

In [9]:
coord_altaz = coord_today.transform_to(
    AltAz(obstime=t_today, location=IITK)
)

In [10]:
print(f"Azimuth = {coord_altaz.az.to(u.deg):.2f}\nElevation = {coord_altaz.alt.to(u.deg):.2f}")

Azimuth = 54.96 deg
Elevation = 57.18 deg


# Question 3
Show in a polar plot, how the azimuth and elevation of this source would change when observing at IITK.

In [11]:
import numpy as np
import plotly.graph_objects as go

In [12]:
times = t_today + np.linspace(0, 24, 145) * u.hour
altaz = coord_today.transform_to(AltAz(obstime=times, location=IITK))

In [13]:
fig = go.Figure(
    go.Scatterpolar(
        theta=altaz.az.to(u.deg).value,
        r=90 - altaz.alt.to(u.deg).value
    )
)
fig.update_layout(
    xaxis_title="Azimuth (deg)",
    yaxis_title="Elevation (deg)",
    title="Alt-Az track of Cygnus A from IIT Kanpur"
)
fig.show()

In a day, the source seems to move in alt az coordinates due to the rotation of Earth

# Question 4
Plot how Sun's RA and Dec changes over a year. Compare this with respect to that of Cygnus over a year?



In [14]:
from astropy.coordinates import get_sun

### Starting from 1 Jan, 2025

In [15]:
times = Time("2025-01-01", scale="utc") + np.arange(365) * u.day
sun = get_sun(times)
cyg = coord_j2000.transform_to(FK5(equinox=times))

In [16]:
fig = go.Figure()
fig.add_trace(
    go.Scatterpolar(
        theta=sun.ra.deg,
        r=90-sun.dec.deg,
        mode="lines",
        name="Sun"
    )
)
fig.add_trace(
    go.Scatterpolar(
        theta=cyg.ra.rad,
        r=90-cyg.dec.deg,
        mode="markers",
        name="Cygnus A"
    )
)
fig.update_layout(
    title="RA–Dec sky track (polar projection)"
)
fig.show()


The Sun seems to trace a path in the Equitorial coordinates map, due to the revolution of Earth around Sun (intersecting dec=0 at Vernal and Spring Equinoxes and dec=90 at the Winter and Solstice), but Cygnus in contrast doesn't have a proper motion, thus doesn't seem to be moving